In [18]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import geopandas as gp
import numpy as np
import random

import pylab as plt
import os

from geopandas.tools import sjoin
from shapely.geometry import Point

from fuzzywuzzy import process

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)

numpy.random.seed(2015)

PARQA = os.getenv('PARQA')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Split calls to named and geolocated

In [34]:
calls = pd.read_csv(PARQA + 'data/311/311DPR.csv', encoding='utf8', na_values='Unspecified')

In [35]:
myCalls = calls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type', 'Complaint Type']]
myCalls['Park Facility Name'] = myCalls['Park Facility Name'].str.lower()
myCalls['Park Facility Name'].head()


namedCalls = myCalls[pd.notnull(myCalls['Park Facility Name'])]
geoCalls = myCalls[(pd.isnull(myCalls['Park Facility Name'])) & (pd.notnull(myCalls.Latitude))]

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
geoCalls.head()

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type
0,NaN,Snow or Ice,12/31/2010 09:04:48 PM,01/03/2011 12:03:59 PM,-73.931120,40.668798,Park,Maintenance or Facility
3,NaN,Snow or Ice,12/31/2010 03:36:37 PM,01/03/2011 09:41:24 AM,-73.962835,40.688556,Park,Maintenance or Facility
4,NaN,Snow or Ice,12/31/2010 03:03:16 PM,01/03/2011 12:15:38 PM,-73.999809,40.636935,Park,Maintenance or Facility
6,NaN,Snow or Ice,12/31/2010 12:59:59 PM,01/03/2011 12:23:04 PM,-73.999456,40.609951,Park,Maintenance or Facility
7,NaN,Snow or Ice,12/31/2010 12:12:02 PM,01/03/2011 12:19:51 PM,-73.977616,40.633153,Park,Maintenance or Facility


## GeoCalls: spatial join with parks to get parkName

In [37]:
parks = gp.read_file(PARQA + 'data/SHP/DPR_ParksProperties_001/DPR_ParksProperties_001.shp')[['geometry','SIGNNAME']]

In [38]:
parks.columns

Index([u'geometry', u'SIGNNAME'], dtype='object')

In [39]:
def toGeoDataFrame(df, lat='Latitude',lon='Longitude'):
    '''dataframe to geodataframe'''
    df['geometry'] = df.apply(lambda z: Point(z[lon], z[lat]), axis=1)
    df = gp.GeoDataFrame(df)
    df.crs = {'init': 'epsg:4326', 'no_defs': True}
    return df 

In [40]:
geoCalls = toGeoDataFrame(geoCalls).to_crs(parks.crs)

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [41]:
geoCalls = sjoin(geoCalls, parks, how="left").to_crs(epsg=4326)

In [48]:
geoCalls = geoCalls[pd.notnull(geoCalls.SIGNNAME)]
geoCalls['Park Facility Name'] = geoCalls['SIGNNAME']
geoCalls = geoCalls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type', 'Complaint Type']]

In [49]:
geoCalls.head()

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type
398,Grand Army Plaza,Rodent Sighting,12/02/2010 10:45:42 AM,12/06/2010 09:34:00 AM,-73.968730,40.673928,Park,Maintenance or Facility
744,Grand Army Plaza,Rodent Sighting,11/11/2010 03:08:10 PM,11/15/2010 09:52:24 AM,-73.969000,40.674713,Park,Maintenance or Facility
1543,Grand Army Plaza,Rodent Sighting,10/03/2010 07:15:35 PM,10/05/2010 09:06:58 AM,-73.970749,40.672638,Park,Maintenance or Facility
2235,Grand Army Plaza,Rodent Sighting,09/11/2010 10:14:20 PM,09/15/2010 10:35:05 AM,-73.968730,40.673928,Park,Maintenance or Facility
2705,Grand Army Plaza,Rodent Sighting,08/31/2010 10:18:33 AM,09/01/2010 04:18:46 PM,-73.968925,40.672937,Park,Maintenance or Facility


In [51]:
calls2 = pd.concat([namedCalls, geoCalls])

In [52]:
calls2.head()

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type
1,geo soilan park - battery park city,Graffiti or Vandalism,12/31/2010 04:31:52 PM,12/31/2010 05:36:58 PM,NaN,NaN,Park,Maintenance or Facility
2,brookville park,Snow or Ice,12/31/2010 04:17:22 PM,01/06/2011 08:58:30 AM,NaN,NaN,Park,Maintenance or Facility
5,highland park,Snow or Ice,12/31/2010 02:57:34 PM,01/03/2011 11:31:26 AM,NaN,NaN,Park,Maintenance or Facility
10,prospect park - east parade grounds,Dead Animal,12/31/2010 11:26:34 AM,01/03/2011 11:12:37 AM,NaN,NaN,Park,Animal in a Park
11,central park - east 96th street playground,Snow or Ice,12/31/2010 11:18:31 AM,01/04/2011 12:11:02 PM,NaN,NaN,Park,Maintenance or Facility


In [53]:
calls2.shape

(55095, 8)

## Now match those names

In [55]:
calls2['Park Facility Name'] = calls2['Park Facility Name'].str.lower()
calls2['Park Facility Name'].value_counts()

central park                                  2554
riverside park                                1208
prospect park                                  961
rockaway beach boardwalk                       761
flushing meadows corona park                   755
beach - brighton                               593
hudson river park                              485
morningside park                               439
tompkins square park                           417
marine park                                    410
forest park                                    409
van cortlandt park                             375
washington square park                         373
mccarren park                                  352
astoria park                                   350
east river park                                325
sunset park                                    324
inwood hill park                               300
inwood hill park - nature center               300
fort tryon park                

## ParkID

In [182]:
prop = pd.read_excel(PARQA + 'data/Input/Parks_Data/CUSP_Adjusted_Spatial_Data.xlsx')[['ParkID','Name']]
prop = prop.dropna()
prop.Name = prop.Name.str.lower()
prop['type']='pid'

In [183]:
def trySplit(x, spl='-'):
    '''get rid of addons'''
    if spl in x:
        return x.split(spl)[0].strip()
    else:
        return x

In [184]:
# prop.ParkID = prop.ParkID.apply(trySplit)
# prop.Name = prop.Name.apply(lambda x: trySplit(x, spl='Zone'))

In [185]:
prop.head()

,ParkID,Name,type
0,M058-07,marcus garvey memorial park,pid
1,M058-06,marcus garvey memorial park,pid
2,M058-01,mt. morris east,pid
3,M047-03,thomas jefferson park,pid
4,M273-01,othmar ammann playground,pid


In [200]:
prop[prop.Name.str.contains('riverside')]

,ParkID,Name,type
502,[M071-ZN10],riverside dr. pedestrian zone b,pid
503,[M071-ZN11],riverside dr. pedestrian zone c,pid
516,[M071-ZN25],riverside dr. pedestrian zone e,pid
520,[M072-ZN26],riverside dr. pedestrian zone f,pid
521,[M072-ZN27],riverside dr. pedestrian zone g,pid
523,[M072-ZN29],riverside / fort washington link,pid
2125,[X336],hunts point riverside park,pid
2152,[M071-ZN12],riverside dr. pedestrian zone d,pid


In [187]:
d = [
    {'ParkID':'B073','type':'abstr','Name': 'prospect park'},
    {'ParkID':'M010','type':'abstr','Name': 'central park'},
    {'ParkID':'Q004','type':'abstr','Name': 'astoria park'},
    {'ParkID':'X010','type':'abstr','Name': 'crotona parks'},
    {'ParkID':'Q162','type':'abstr', 'Name': 'rockaway beach boardwalk'},
    {'ParkID':'M014','type':'abstr', 'Name': 'jackie robinson park'},
    {'ParkID':'M028','type':'abstr', 'Name': 'fort washington park'},
    {'ParkID':'M037','type':'abstr', 'Name': 'highbridge park'},
    {'ParkID':'M098','type':'abstr', 'Name': 'washington square park'},
    {'ParkID':'M105','type':'abstr', 'Name': 'sara d. roosevelt park'},
    {'ParkID':'M107','type':'abstr', 'Name': "hell's kitchen park"},
    {'ParkID':'M283','type':'abstr', 'Name': 'battery park city'},
    {'ParkID':'Q001','type':'abstr', 'Name': 'alley pond park'},
    {'ParkID':'Q005','type':'abstr', 'Name': 'baisley pond park'},
    {'ParkID':'Q009','type':'abstr', 'Name': 'macneil park'},
    {'ParkID':'Q012','type':'abstr', 'Name': 'crocheron park'},
    {'ParkID':'Q020','type':'abstr', 'Name': 'highland park'},
    {'ParkID':'Q021','type':'abstr', 'Name': 'cunningham park'},
    {'ParkID':'Q024','type':'abstr', 'Name': 'kissena park'},
    {'ParkID':'Q102','type':'abstr', 'Name': 'juniper valley park'},
    {'ParkID':'R129','type':'abstr', 'Name': 'greenbelt native plant center'},
    {'ParkID':'B058','type':'abstr', 'Name': 'mccarren park'},
    
    
] 

In [188]:
# prop = prop.append(pd.DataFrame(d), ignore_index=1)
abstr = pd.DataFrame(d)

In [189]:
# prop[prop.type=='abstr']

def getIDList(ID):
    '''get list of pID for this general structure'''
    return prop.ParkID[prop.ParkID.str.startswith(ID)].tolist()

In [190]:
abstr.ParkID = abstr.ParkID.apply(getIDList)

In [191]:
abstr.head()

,Name,ParkID,type
0,prospect park,"[B073-02D, B073-02, B073-10, B073-20, B073-09,...",abstr
1,central park,"[M010-151, M010-089, M010-090, M010-143, M010-...",abstr
2,astoria park,"[Q004-01, Q004-ZN01, Q004-ZN02, Q004-ZN03, Q004A]",abstr
3,crotona parks,"[X010-10, X010-05, X010-12, X010-08, X010-03, ...",abstr
4,rockaway beach boardwalk,"[Q162-ZN01A, Q162-ZN01B, Q162-ZN02A, Q162-ZN02...",abstr


In [192]:
# prop.groupby('ParkID').agg(lambda x: x.tolist())
prop.ParkID = prop.ParkID.apply(lambda x: [x])

In [193]:
prop.head()

,ParkID,Name,type
0,[M058-07],marcus garvey memorial park,pid
1,[M058-06],marcus garvey memorial park,pid
2,[M058-01],mt. morris east,pid
3,[M047-03],thomas jefferson park,pid
4,[M273-01],othmar ammann playground,pid


In [194]:
prop2 = pd.concat((prop, abstr))

## Check how ontology works

In [195]:
onto = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/onto_data/Ontology_matched.csv', index_col=0)[['cleanName','NAME']]
onto.rename(columns={'NAME':'newName','cleanName':'Name'}, inplace=1)
onto.head()

,Name,newName
0,geo soilan park - battery park city,battery park city
1,geo soilan park - battery park city,battery park city
2,brookville park,brookville park
3,highland park,highland park
4,highland park,highland park


In [196]:
onto2 = onto1.merge(onto, on='Name', how='left')
onto2.newName[pd.isnull(onto2.newName)] = onto2.Name[pd.isnull(onto2.newName)]

In [197]:
### edit Onto
onto2.newName[onto2.Name.str.contains('mccarren park')] = 'mccarren park'

In [198]:
onto2[onto2.newName.str.so]

,Name,newName
0,geo soilan park - battery park city,battery park city
1,geo soilan park - battery park city,battery park city
2,brookville park,brookville park
3,highland park,highland park
4,highland park,highland park
5,highland park,highland park
6,highland park,highland park
7,prospect park - east parade grounds,prospect park
8,central park - east 96th street playground,central park
9,washington hall park,washington hall park


In [199]:
print len(onto1)
x = onto2.merge(prop2[['Name','type']], how='left', left_on='newName', right_on='Name')

print len(x[pd.isnull(x['type'])])
x[pd.isnull(x['type'])]

1671
568


,Name_x,newName,Name_y,type
15,col david marcus playground,col david marcus mem playground,NaN,NaN
24,forest park - giovannelli playground,forest park,NaN,NaN
36,hoover manton playground,manton playground,NaN,NaN
40,carl schurz park,carl schurz park,NaN,NaN
48,doctor gertrude b kelly playground,dr gertrude b kelly playground,NaN,NaN
61,brooklyn civic center parks,brooklyn civic center parks,NaN,NaN
62,may matthews playground,may matthews playground,NaN,NaN
63,cuyler gore playground,cuyler gore,NaN,NaN
69,terrapin playground - jhs 51,terrapin playground,NaN,NaN
70,terrapin playground - jhs 51,terrapin playground,NaN,NaN
